This experiment is done on the ground truth. The ground truth is created based on the NMVW constituents' display names and the rdfs:labels retrieved from the wikidata. It only contains constituesnts where the wikidata identifies is known.

In [1]:
import sys
sys.path.append("..") 

In [2]:
import pickle, pandas
from data_matching import exact_string_match, match_lastName, match_with_abbreviation, match_fuzzy_string
from calculate_result import calculate_result

/Users/sarah_shoilee/opt/anaconda3/envs/py39deezy/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
with open('data/ground_truth.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
candidates = set([str(item) for _, row in data.iterrows() for item in row['wiki_label']])

In [5]:
print(f"The size of ground truth: {len(data.index)} and size of candidates: {len(list(set(candidates)))}")

The size of ground truth: 6178 and size of candidates: 11501


# Exact string matching

In [6]:
# Exact string matching
result_exact = exact_string_match(data, candidates)
exact_match = len(result_exact.loc[result_exact["match"] == "YES"])
print(f"The number of positive match is {exact_match}")

The number of positive match is 3360


In [7]:
with open("results/NMVWtoWikiExactMatchResults.pkl", "wb") as handle:
    pickle.dump(result_exact, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
df = pandas.read_pickle("results/NMVWtoWikiExactMatchResults.pkl")
print(len(df.index))

6178


In [9]:
calculate_result("results/NMVWtoWikiExactMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 3346
Recall: 0.5415992230495306


Total query: 6178
Total retrieved: 3360
Correct correspondence count: 3346 
Precision: 0.9958333333333333


F-measure: 0.7016145942545607



(6178, 3360, 3346, 0.5415992230495306, 0.9958333333333333, 0.7016145942545607)

# Surname Matching

In [10]:
result_surname = match_lastName(data, candidates)
surname_match = len(result_surname.loc[result_surname["match"] == "YES"])
print(f"The number of positive match is {surname_match}")

The number of positive match is 5245


In [11]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result_surname, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

6178


In [13]:
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 4991
Recall: 0.8078666235027517


Total query: 6178
Total retrieved: 9613
Correct correspondence count: 5833 
Precision: 0.6067824820555497


F-measure: 0.6930330822714871



(6178, 9613, 5833, 0.8078666235027517, 0.6067824820555497, 0.6930330822714871)

# Abbreviation Matching

In [14]:
result_abbreviation = match_with_abbreviation(data, candidates)
abbreviationlen_match = len(result_abbreviation.loc[result_abbreviation["match"] == "YES"])
print(f"The number of positive match is {abbreviationlen_match}")

The number of positive match is 3662


In [15]:
with open("results/NMVWtoWikiAbbreviationMatchResults.pkl", "wb") as handle:
    pickle.dump(result_abbreviation, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
df = pandas.read_pickle("results/NMVWtoWikiAbbreviationMatchResults.pkl")
print(len(df.index))

6178


In [17]:
calculate_result("results/NMVWtoWikiAbbreviationMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 766
Recall: 0.12398834574295889


Total query: 6178
Total retrieved: 3662
Correct correspondence count: 766 
Precision: 0.2091753140360459


F-measure: 0.15569105691056911



(6178, 3662, 766, 0.12398834574295889, 0.2091753140360459, 0.15569105691056911)

# Fuzzy String Match

In [6]:
result_fuzzymatch = match_fuzzy_string(data, candidates, max_score=75)
fuzzy_match = len(result_fuzzymatch.loc[result_fuzzymatch["match"] == "YES"])
print(f"The number of positive match is {fuzzy_match}")

6178it [2:07:10,  1.24s/it]

The number of positive match is 6120


In [7]:
with open("results/NMVWtoWikiFuzzyStringMatchResults.pkl", "wb") as handle:
    pickle.dump(result_fuzzymatch, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
df = pandas.read_pickle("results/NMVWtoWikiFuzzyStringMatchResults.pkl")
print(len(df.index))

6178


In [9]:
calculate_result("results/NMVWtoWikiFuzzyStringMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 5501
Recall: 0.8904176108773065


Total query: 6178
Total retrieved: 16486
Correct correspondence count: 6455 
Precision: 0.391544340652675


F-measure: 0.5439131417908255



(6178, 16486, 6455, 0.8904176108773065, 0.391544340652675, 0.5439131417908255)